# **Question 1** - Pseudo-Hamiltonian for the Simple Harmonic Oscillator

## **a)** Conservation of the Pseudo-Hamiltonian

Use the Jacobian for the leapfrog method to show that the pseudo-Hamiltonian
$$\tilde{H}(h) = \frac{1}{2}x^2 + \frac{1}{2}v^2\frac{1}{1 - h^2/4}$$
is conserved when using the leapfrog method.

For simplicity, the mass, spring constant, and oscillator frequencies are set to 1 here.

The Leapfrog method is:

$$v_{n+1/2} = v_n + a_n \frac{\Delta t}{2} = v_n - x_n \frac{\Delta t}{2}$$
$$x_{n+1/2} = x_n + v_{n + 1/2} \Delta t$$
$$v_{n+1} = v_{n + 1/2} + a_{n + 1} \frac{\Delta t}{2} = v_{n + 1/2} - x_{n + 1} \frac{\Delta t}{2}$$

which can be written in Matrix form as:
$$
\begin{pmatrix}
x_{n+1}\\
v_{n+1}
\end{pmatrix} =
\begin{pmatrix}
1 & 0\\
-\frac{h}{2} & 1
\end{pmatrix}
\begin{pmatrix}
1 & h\\
0 & 1
\end{pmatrix}
\begin{pmatrix}
1 & 0\\
-\frac{h}{2} & 1
\end{pmatrix}
\begin{pmatrix}
x_{n}\\
v_{n}
\end{pmatrix}
$$

In [2]:
import sympy.matrices as sm
import sympy

def Leapfrog(dt):
    lf1 = sm.Matrix([[1, 0], [-dt/2, 1]])
    lf2 = sm.Matrix([[1, dt], [0, 1]])
    Jlf = lf1*lf2*lf1
    return Jlf

# declare the step size h as a symbol
h = sympy.Symbol('h')
print(sympy.simplify(Leapfrog(h)))

Matrix([[1 - h**2/2, h], [h**3/4 - h, 1 - h**2/2]])


## **b)** Implemenation of the Leapfrog method

Implement the leapfrog method for the simple harmonic oscillator numerically, and verify that they indeed do conserve $\tilde{H}$.

## **c)** Scaling of Energy Error

Use the code from part b to calculate the scaling of the energy error with the number of steps per oscillation cycle for integrations of:
1. A whole oscillation
2. Half an oscillation

Discuss whether the escalins you find agree with the analytic results for the leapfrog method.